In [1]:
import torch
import torch_geometric as pyg
import pandas as pd
import networkx as nx
import pickle
import numpy as np
from torch_sparse.tensor import SparseTensor

In [51]:
All_Edges_Graph = pd.read_csv('data/HeriGraph/raw/dataset/Venice/Edge_List.csv', sep='\t', index_col='Unnamed: 0')

In [56]:
G4 = nx.from_pandas_edgelist(All_Edges_Graph.loc[:,['0','1']], source='0', target='1')
G3 = nx.from_pandas_edgelist(All_Edges_Graph[All_Edges_Graph.Temporal_Similarity>0].loc[:,['0','1']],source='0', target='1')

In [61]:
set(G3.nodes) == set(G4.nodes)

True

In [64]:
attr_lables = pd.read_csv('data/HeriGraph/raw/dataset/Venice/Attribute_Labels.csv', sep='\t', index_col='Unnamed: 0')
v_feature = pd.read_csv('data/HeriGraph/raw/dataset/Venice/Visual_Features.csv', sep='\t', index_col='Unnamed: 0')

In [67]:
for id1, id2 in zip(v_feature["ID"], attr_lables["ID"]):
    if id1 != id2:
        print(id1 == id2)

In [59]:
pyg3 = pyg.utils.from_networkx(G3)
pyg3

Data(edge_index=[2, 442828], data1=[2951, 3], data_2=[2951, 99], num_nodes=2951)

In [13]:
value_labels = pd.read_csv('data/HeriGraph/Venice/Value_Labels.csv', sep='\t', index_col='Unnamed: 0')

In [15]:
value_labels = value_labels.to_numpy()

In [24]:
logits = value_labels[:, 2:13].astype(np.float32)

In [67]:
label = np.argmax(logits,axis = 1)

In [79]:
np.max(logits, axis = 1)

array([0.3314337 , 0.42099845, 0.35036433, ..., 0.3184082 , 0.3184082 ,
       0.3184082 ], dtype=float32)

In [2]:
from fedframe.dataset.dataset import HeriGraph

In [3]:
data = HeriGraph("./data/HeriGraph", "Suzhou/", 3)

In [4]:
from fedframe.dataset.sampler import HeriSplit, HeriLoader

In [5]:
ds = HeriSplit(data, 3, train = 0.2, test = 0.5)

In [9]:
dsl = HeriLoader(ds, train = True, GAT = True)

In [19]:
idx_1 = dsl.sample_batch(16)
idx_2 = dsl.sample_id(torch.tensor(idx_1).to(torch.int64), torch.tensor(3).to(torch.int64), 4)

D:\Dropbox\Research\Projects\Federated Learning\Experiments\python\GNN\fedframe\dataset\sampler.py:222: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  adj_t, new_node = self.adj.sample_adj(torch.tensor(output), batch_size, replace = False)


In [23]:
adj, idx = dsl.sample_graph(idx_2)

In [28]:
pyg.utils.is_undirected(adj[-1])

True

In [10]:
dsl.test_set[0:0]

[]

In [5]:
isinstance(data.data.edge_index, SparseTensor)

False

In [7]:
adj = SparseTensor(row = data.data.edge_index[0], col=data.data.edge_index[1], value = None, sparse_sizes = (data.data.num_nodes, data.data.num_nodes)).t()

In [8]:
adj.storage.rowptr()

tensor([     0,    466,    932,  ..., 586430, 586543, 586656])

In [10]:
adj_t, new_id = adj.sample_adj(torch.tensor([0,2,4,6]), 2, replace = False)

In [13]:
ni = new_id.numpy()

In [15]:
ni.sort()

In [16]:
ni

array([  0,   2,   4,   6,  25,  31,  37,  86, 141, 317, 370, 946])